# Transfer Learning CIFAR10

* Train a simple convnet on the CIFAR dataset the first 5 output classes [0..4].
* Freeze convolutional layers and fine-tune dense layers for the last 5 ouput classes [5..9].


### 1. Import CIFAR10 data and create 2 datasets with one dataset having classes from 0 to 4 and other having classes from 5 to 9 

In [1]:
from keras.datasets import cifar10
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dropout
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix

Using TensorFlow backend.


In [2]:
df_cifar10 = cifar10.load_data()

In [3]:
(train_df), (test_df) = cifar10.load_data()

In [4]:
x_train_04 = []
x_train_59 = []
y_train_04 = []
y_train_59 = []

for idx, item in enumerate(train_df[1]):
    if item[0] in [0,1,2,3,4]:
        x_train_04.append(train_df[0][idx])
        y_train_04.append(item[0])
    elif item[0] in [5,6,7,8,9]:
        x_train_59.append(train_df[0][idx])
        y_train_59.append(item[0])
x_train_04 = np.array(x_train_04)
x_train_59 = np.array(x_train_59)

In [5]:
x_test_04 = []
x_test_59 = []
y_test_04 = []
y_test_59 = []

for idx, item in enumerate(test_df[1]):
    if item[0] in [0,1,2,3,4]:
        x_test_04.append(test_df[0][idx])
        y_test_04.append(item[0])
    elif item[0] in [5,6,7,8,9]:
        x_test_59.append(test_df[0][idx])
        y_test_59.append(item[0])
x_test_04 = np.array(x_test_04)
x_test_59 = np.array(x_test_59)

### 2. Use One-hot encoding to divide y_train and y_test into required no of output classes

In [8]:
y_train_04

[4,
 1,
 1,
 2,
 3,
 4,
 2,
 3,
 2,
 4,
 3,
 2,
 3,
 4,
 0,
 0,
 1,
 3,
 4,
 0,
 3,
 3,
 3,
 2,
 2,
 1,
 1,
 1,
 2,
 2,
 0,
 2,
 2,
 2,
 4,
 3,
 1,
 1,
 2,
 1,
 1,
 4,
 3,
 1,
 0,
 3,
 1,
 3,
 4,
 4,
 4,
 2,
 3,
 0,
 1,
 1,
 1,
 4,
 1,
 3,
 1,
 2,
 1,
 0,
 0,
 1,
 2,
 2,
 2,
 1,
 0,
 4,
 1,
 1,
 1,
 2,
 1,
 3,
 3,
 2,
 4,
 4,
 3,
 4,
 4,
 3,
 1,
 4,
 0,
 1,
 3,
 2,
 3,
 4,
 1,
 0,
 4,
 1,
 0,
 0,
 2,
 2,
 3,
 0,
 1,
 3,
 1,
 3,
 1,
 0,
 2,
 0,
 0,
 1,
 1,
 3,
 0,
 1,
 1,
 3,
 4,
 1,
 3,
 3,
 4,
 1,
 1,
 3,
 1,
 1,
 4,
 0,
 3,
 4,
 2,
 4,
 1,
 0,
 0,
 2,
 1,
 2,
 0,
 3,
 2,
 0,
 4,
 4,
 2,
 1,
 1,
 2,
 1,
 0,
 4,
 1,
 1,
 3,
 3,
 0,
 1,
 1,
 1,
 3,
 0,
 3,
 3,
 2,
 4,
 0,
 3,
 4,
 0,
 4,
 4,
 0,
 0,
 0,
 1,
 2,
 2,
 4,
 1,
 3,
 3,
 0,
 4,
 0,
 1,
 0,
 3,
 4,
 4,
 2,
 3,
 1,
 4,
 0,
 1,
 3,
 1,
 4,
 4,
 2,
 0,
 2,
 2,
 0,
 0,
 0,
 2,
 4,
 0,
 3,
 0,
 4,
 2,
 2,
 2,
 1,
 4,
 1,
 4,
 4,
 0,
 0,
 3,
 3,
 4,
 0,
 4,
 0,
 1,
 0,
 0,
 4,
 1,
 2,
 1,
 0,
 0,
 2,
 3,
 4,
 3,
 0,
 1,
 2,
 4,
 1,


In [6]:
y_train_04_one_hot = pd.get_dummies(np.array(y_train_04))
y_train_59_one_hot = pd.get_dummies(np.array(y_train_59))
y_test_04_one_hot = pd.get_dummies(np.array(y_test_04))
y_test_59_one_hot = pd.get_dummies(np.array(y_test_59))

In [18]:
y_train_04_one_hot.head()

,0,1,2,3,4
0,0,0,0,0,1
1,0,1,0,0,0
2,0,1,0,0,0
3,0,0,1,0,0
4,0,0,0,1,0


### 3. Build a sequential neural network model which can classify the classes 0 to 4 of CIFAR10 dataset with at least 80% accuracy on test data

In [11]:
x_train_04.shape[1:]

(32, 32, 3)

In [12]:
model = Sequential()

model.add(Conv2D(32, (3,3), activation='relu', input_shape= x_train_04.shape[1:]))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.20))

model.add(Conv2D(64,(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(GlobalAveragePooling2D())


model.add(Dense(128, activation='relu'))
model.add(Dropout(0.20))

model.add(Dense(5, activation='softmax'))

model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 13, 13, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 6, 6, 32)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 6, 6, 32)          0         
_________________________________________________________________
conv2d_3 (Conv2D)    

In [13]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [14]:
X_train_04_scratch = x_train_04/255
X_test_04_scratch = x_test_04/255

In [15]:
X_train_04_scratch.shape

(25000, 32, 32, 3)

In [16]:
y_train_04_one_hot.shape

(25000, 5)

In [17]:
y_train_04_one_hot

,0,1,2,3,4
0,0,0,0,0,1
1,0,1,0,0,0
2,0,1,0,0,0
3,0,0,1,0,0
4,0,0,0,1,0
5,0,0,0,0,1
6,0,0,1,0,0
7,0,0,0,1,0
8,0,0,1,0,0
9,0,0,0,0,1


In [11]:
#Fitting the model on the train data and labels.
model.fit(X_train_04_scratch, y_train_04_one_hot, batch_size=32, epochs=50, 
          verbose=1, validation_split=0.2, shuffle=True)

Instructions for updating:
Use tf.cast instead.
Train on 20000 samples, validate on 5000 samples
Epoch 1/50
20000/20000 [==============================] - 11s 563us/step - loss: 1.2531 - acc: 0.4357 - val_loss: 1.0622 - val_acc: 0.5590
Epoch 2/50
20000/20000 [==============================] - 12s 601us/step - loss: 1.0363 - acc: 0.5672 - val_loss: 0.9432 - val_acc: 0.6258
Epoch 3/50
20000/20000 [==============================] - 12s 608us/step - loss: 0.9517 - acc: 0.6137 - val_loss: 0.8908 - val_acc: 0.6482
Epoch 4/50
20000/20000 [==============================] - 12s 601us/step - loss: 0.8901 - acc: 0.6439 - val_loss: 0.8314 - val_acc: 0.6616
Epoch 5/50
20000/20000 [==============================] - 12s 611us/step - loss: 0.8357 - acc: 0.6713 - val_loss: 0.7891 - val_acc: 0.6940
Epoch 6/50
20000/20000 [==============================] - 12s 618us/step - loss: 0.7964 - acc: 0.6880 - val_loss: 0.7305 - val_acc: 0.7234
Epoch 7/50
20000/20000 [==============================] - 13s 650us/s

In [12]:
#Evaluate the model on the test data
score = model.evaluate(X_test_04_scratch, y_test_04_one_hot)

#Accuracy on test data
print('Accuracy on the Test Images: ', score[1])

5000/5000 [==============================] - 1s 162us/step
Accuracy on the Test Images:  0.8068


In [22]:
X_train_59_scratch = x_train_59/255
X_test_59_scratch = x_test_59/255

### 4. In the model which was built above (for classification of classes 0-4 in CIFAR10), make only the dense layers to be trainable and conv layers to be non-trainable

In [26]:
transf_model = model

In [29]:
initial_weights = model.get_weights()

In [30]:
transf_model.set_weights(initial_weights)

In [33]:
for layer in transf_model.layers[:-3]:
    layer.trainable = False

In [34]:
for layer in transf_model.layers:
    print(layer, layer.trainable)

<keras.layers.convolutional.Conv2D object at 0xb275a6da0> False
<keras.layers.pooling.MaxPooling2D object at 0xb275a6f60> False
<keras.layers.convolutional.Conv2D object at 0xb3433b3c8> False
<keras.layers.pooling.MaxPooling2D object at 0xb2758acc0> False
<keras.layers.core.Dropout object at 0xb3f455f28> False
<keras.layers.convolutional.Conv2D object at 0xb3f455eb8> False
<keras.layers.pooling.MaxPooling2D object at 0xb3f455ef0> False
<keras.layers.core.Dropout object at 0xb3f37bbe0> False
<keras.layers.pooling.GlobalAveragePooling2D object at 0xb3f37bd68> False
<keras.layers.core.Dense object at 0xb3f3aa6d8> True
<keras.layers.core.Dropout object at 0xb3f48d550> True
<keras.layers.core.Dense object at 0xb3f4f1e10> True


### 5. Utilize the the model trained on CIFAR 10 (classes 0 to 4) to classify the classes 5 to 9 of CIFAR 10  (Use Transfer Learning) <br>
Achieve an accuracy of more than 85% on test data

In [35]:
transf_model.fit(X_train_59_scratch, y_train_59_one_hot, batch_size=32, epochs=50, 
          verbose=1, validation_split=0.2, shuffle=True)

Train on 20000 samples, validate on 5000 samples
Epoch 1/50
  256/20000 [..............................] - ETA: 10s - loss: 4.3165 - acc: 0.1133

/Users/mac/anaconda3/lib/python3.7/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


20000/20000 [==============================] - 12s 581us/step - loss: 1.0759 - acc: 0.5846 - val_loss: 0.7515 - val_acc: 0.7244
Epoch 2/50
20000/20000 [==============================] - 12s 608us/step - loss: 0.7712 - acc: 0.7079 - val_loss: 0.6081 - val_acc: 0.7746
Epoch 3/50
20000/20000 [==============================] - 13s 626us/step - loss: 0.6703 - acc: 0.7484 - val_loss: 0.5637 - val_acc: 0.7822
Epoch 4/50
20000/20000 [==============================] - 14s 680us/step - loss: 0.6076 - acc: 0.7727 - val_loss: 0.5138 - val_acc: 0.8178
Epoch 5/50
20000/20000 [==============================] - 13s 654us/step - loss: 0.5731 - acc: 0.7904 - val_loss: 0.4956 - val_acc: 0.8170
Epoch 6/50
20000/20000 [==============================] - 13s 648us/step - loss: 0.5372 - acc: 0.8045 - val_loss: 0.4745 - val_acc: 0.8218
Epoch 7/50
20000/20000 [==============================] - 13s 651us/step - loss: 0.5092 - acc: 0.8118 - val_loss: 0.4434 - val_acc: 0.8332
Epoch 8/50
20000/20000 [==============

In [37]:
#Evaluate the model on the test data
score = transf_model.evaluate(X_test_59_scratch, y_test_59_one_hot)

#Accuracy on test data
print('Accuracy on the Test Images: ', score[1])

5000/5000 [==============================] - 1s 151us/step
Accuracy on the Test Images:  0.8852


# Text classification using TF-IDF

### 6. Load the dataset from sklearn.datasets

In [132]:
from sklearn.datasets import fetch_20newsgroups
import nltk
import re
from nltk.stem.snowball import SnowballStemmer

In [127]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/mac/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [133]:
stemmer = SnowballStemmer("english")

In [39]:
categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']

### 7. Training data

In [40]:
twenty_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)

### 8. Test data

In [41]:
twenty_test = fetch_20newsgroups(subset='test', categories=categories, shuffle=True, random_state=42)

###  a.  You can access the values for the target variable using .target attribute 
###  b. You can access the name of the class in the target variable with .target_names


In [42]:
twenty_train.target

array([1, 1, 3, ..., 2, 2, 2])

In [43]:
twenty_train.target_names

['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']

In [44]:
twenty_train.data[0:5]

['From: sd345@city.ac.uk (Michael Collier)\nSubject: Converting images to HP LaserJet III?\nNntp-Posting-Host: hampton\nOrganization: The City University\nLines: 14\n\nDoes anyone know of a good way (standard PC application/PD utility) to\nconvert tif/img/tga files into LaserJet III format.  We would also like to\ndo the same, converting to HPGL (HP plotter) files.\n\nPlease email any response.\n\nIs this the correct group?\n\nThanks in advance.  Michael.\n-- \nMichael Collier (Programmer)                 The Computer Unit,\nEmail: M.P.Collier@uk.ac.city                The City University,\nTel: 071 477-8000 x3769                      London,\nFax: 071 477-8565                            EC1V 0HB.\n',
 "From: ani@ms.uky.edu (Aniruddha B. Deglurkar)\nSubject: help: Splitting a trimming region along a mesh \nOrganization: University Of Kentucky, Dept. of Math Sciences\nLines: 28\n\n\n\n\tHi,\n\n\tI have a problem, I hope some of the 'gurus' can help me solve.\n\n\tBackground of the probl

### 9.  Now with dependent and independent data available for both train and test datasets, using TfidfVectorizer fit and transform the training data and test data and get the tfidf features for both

In [123]:
def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems

In [213]:
from sklearn.feature_extraction.text import TfidfVectorizer

#define vectorizer parameters
tfidf_vectorizer1 = TfidfVectorizer(tokenizer=tokenize_and_stem, stop_words='english', max_features=980)
tfidf_vectorizer2 = TfidfVectorizer(tokenizer=tokenize_and_stem, stop_words='english', max_features=980)

%time tfidf_train_matrix = tfidf_vectorizer1.fit_transform(twenty_train.data) #fit the vectorizer to synopses
%time tfidf_test_matrix = tfidf_vectorizer2.fit_transform(twenty_test.data) #fit the vectorizer to synopses

print(tfidf_train_matrix.shape)
print(tfidf_test_matrix.shape)

CPU times: user 15.5 s, sys: 16.8 ms, total: 15.5 s
Wall time: 15.6 s
CPU times: user 11.3 s, sys: 8.99 ms, total: 11.3 s
Wall time: 11.3 s
(2257, 980)
(1502, 980)


### 10. Use logisticRegression with tfidf features as input and targets as output and train the model and report the train and test accuracy score

In [222]:
logi_model = LogisticRegression()

In [223]:
logi_model.fit(tfidf_train_matrix, twenty_train.target)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [224]:
logi_model.score(tfidf_train_matrix, twenty_train.target)

0.973859105006646

In [225]:
logi_model.score(tfidf_test_matrix, twenty_test.target)

0.4786950732356858